# AI Backend Refactor Assistant  
**Non-agentic LLM-powered tool to transform Python backend logic into production-ready artifacts**

**Goal**  
Convert any Python function (or existing FastAPI endpoint) into:

- 🦀 High-performance Rust implementation  
- 📝 Enhanced Python version (with full docstrings + type hints)  
- 🧪 Comprehensive pytest unit tests  
- 🌐 Automatic FastAPI wrapper (when input is a plain function)  
- 📘 Valid OpenAPI 3.0 specification (YAML)

Built with Gradio UI • Uses frontier LLMs via OpenRouter/Groq/etc. • Sequential pipeline (no agents)


In [7]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display
import re


In [10]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")



OpenAI API Key not set
Anthropic API Key not set (and this is optional)
Google API Key exists and begins AI
Grok API Key not set (and this is optional)
Groq API Key exists and begins gsk_
OpenRouter API Key exists and begins sk-or-


In [ ]:
# Connect to client libraries

# openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"

openai = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
anthropic = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)



In [12]:
models = ["gpt-5", "claude-sonnet-4-5-20250929", "grok-4", "gemini-2.5-pro", "qwen2.5-coder", "deepseek-coder-v2", "gpt-oss:20b", "qwen/qwen3-coder-30b-a3b-instruct", "openai/gpt-oss-120b", ]

clients = {"gpt-5": openai, "claude-sonnet-4-5-20250929": anthropic, "grok-4": grok, "gemini-2.5-pro": gemini, "openai/gpt-oss-120b": groq, "qwen2.5-coder": ollama, "deepseek-coder-v2": ollama, "gpt-oss:20b": ollama, "qwen/qwen3-coder-30b-a3b-instruct": openrouter}

# Want to keep costs ultra-low? Replace this with models of your choice, using the examples from yesterday

In [21]:
python_hard = """# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [16]:
def get_rust_messages(python_code: str) -> list:
    system = """You are a world-class Rust performance engineer.
Rewrite the Python backend logic as a complete, high-performance, zero-dependency `main.rs`.
Rules:
- Use ONLY the Rust standard library
- Maximize runtime speed (opt-level 3 mindset: native CPU, minimal allocations, best algorithms)
- Include a `fn main()` that runs the logic and prints the same output the Python code would produce
- Add comprehensive `///` documentation
- The code must compile with `rustc main.rs -C opt-level=3 -C target-cpu=native`
Respond **exclusively** with the full Rust source code. No explanations."""
    user = f"Python code to port:\n```python\n{python_code}\n```"
    return [{"role": "system", "content": system}, {"role": "user", "content": user}]


def get_enhanced_python_messages(python_code: str) -> list:
    system = """You are a senior Python engineer.
Enhance the provided code WITHOUT changing any logic:
- Add complete Google-style docstrings to every function and class
- Add/fix all type hints (use typing when needed)
- Add clear inline comments for non-obvious sections
- Improve readability where possible
Respond **exclusively** with the full enhanced Python code."""
    user = f"Code to enhance:\n```python\n{python_code}\n```"
    return [{"role": "system", "content": system}, {"role": "user", "content": user}]


def get_tests_messages(python_code: str) -> list:
    system = """You are a pytest expert.
Generate a complete, ready-to-run pytest test file for the provided Python code.
Requirements:
- Happy-path, edge-case, and error-case tests
- Use `@pytest.mark.parametrize` wherever sensible
- No external mocks unless absolutely required
- Tests should be importable (assume the original code is in `backend.py`)
Respond **exclusively** with the full pytest Python code."""
    user = f"Generate tests for:\n```python\n{python_code}\n```"
    return [{"role": "system", "content": system}, {"role": "user", "content": user}]


def get_fastapi_wrapper_messages(python_code: str) -> list:
    system = """You are a FastAPI expert.
If the input is a standalone Python function → wrap it as a complete minimal FastAPI application.
If the input is already a FastAPI endpoint/app → return it unchanged (but guarantee it is a full runnable app with `app = FastAPI()` and uvicorn block).

Rules for standalone function:
- Use POST endpoint named after the function (or `/compute`)
- Create Pydantic `BaseModel` for request body when parameters exist
- Return JSON `{"result": ...}`
- Include `if __name__ == "__main__": uvicorn.run(...)`
- Keep the original computation logic untouched

Respond **exclusively** with the full FastAPI Python code."""
    user = f"Code:\n```python\n{python_code}\n```"
    return [{"role": "system", "content": system}, {"role": "user", "content": user}]


def get_openapi_messages(fastapi_code: str) -> list:
    system = """You are an OpenAPI 3.0 specialist.
From the provided FastAPI code, generate a complete, valid OpenAPI 3.0 YAML specification.
Include paths, requestBody schemas, response schemas, and status codes.
Respond **exclusively** with the YAML content."""
    user = f"FastAPI code:\n```python\n{fastapi_code}\n```"
    return [{"role": "system", "content": system}, {"role": "user", "content": user}]

In [17]:
def call_llm(messages: list, model: str) -> str:
    client = clients.get(model, openai)          # fallback to OpenRouter client
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.1,
            max_tokens=8192,
        )
        content = response.choices[0].message.content.strip()

        # Extract first code block (most reliable for clean output)
        code_match = re.search(r'```(?:\w+)?\s*(.*?)```', content, re.DOTALL)
        if code_match:
            content = code_match.group(1).strip()
        return content
    except Exception as e:
        return f"❌ LLM Error ({model}): {str(e)}"

In [29]:
def generate_artifacts(python_code: str, selected_model: str):
    if not python_code or not python_code.strip():
        empty = "❌ No input code provided."
        return empty, empty, empty, empty, empty

    # 1. Enhanced Python (always)
    enhanced_python = call_llm(get_enhanced_python_messages(python_code), selected_model)

    # 2. FastAPI Wrapper (LLM decides if input is already an endpoint)
    fastapi_wrapper = call_llm(get_fastapi_wrapper_messages(python_code), selected_model)

    # 3. High-performance Rust
    rust_code = call_llm(get_rust_messages(python_code), selected_model)
    rust_display = f"```rust\n{rust_code}\n```" if rust_code else "No Rust code generated."

    # 4. Pytest unit tests
    unit_tests = call_llm(get_tests_messages(python_code), selected_model)

    # 5. OpenAPI 3.0 YAML (based on the wrapper)
    openapi_spec = call_llm(get_openapi_messages(fastapi_wrapper), selected_model)

    
    return rust_display, enhanced_python, unit_tests, fastapi_wrapper, openapi_spec

In [ ]:
from styles import CSS

with gr.Blocks(
    css=CSS if "CSS" in globals() else None,
    theme=gr.themes.Monochrome(),
    title="AI Backend Refactor Assistant"
) as ui:
    gr.Markdown("""
    # 🧠 AI Backend Refactor Assistant
    **Non-agentic** • Production-ready artifacts from any Python backend function or FastAPI endpoint
    """)

    with gr.Row(equal_height=True):
        with gr.Column(scale=5):
            python_input = gr.Code(
                label="📥 Python Input (standalone function or @app.get/post endpoint)",
                value=python_hard,
                language="python",
                lines=22
            )
        with gr.Column(scale=1):
            model_dropdown = gr.Dropdown(
                choices=models,
                value=models[0],
                label="🤖 Model (used for ALL tasks)",
                info="All 5 artifacts are generated sequentially with this model"
            )
            file_upload = gr.File(
                label="📁 Upload .py file",
                file_types=[".py"],
                height=100
            )

    generate_btn = gr.Button("🚀 Generate All 5 Artifacts", variant="primary", size="large")

    # Collapsible accordions exactly as specified in the SRD
    with gr.Column():
        with gr.Accordion("🦀 High-Performance Rust Implementation", open=True):
            rust_out = gr.Markdown(value="```rust\n// Waiting for generation...\n```")

        with gr.Accordion("📝 Enhanced Python (docstrings + types)", open=False):
            enhanced_out = gr.Code(label="", language="python", lines=25)

        with gr.Accordion("🧪 Pytest Unit Tests", open=False):
            tests_out = gr.Code(label="", language="python", lines=25)

        with gr.Accordion("🌐 FastAPI Wrapper (auto-generated if standalone)", open=False):
            fastapi_out = gr.Code(label="", language="python", lines=25)

        with gr.Accordion("📘 OpenAPI 3.0 Specification (YAML)", open=False):
            openapi_out = gr.Code(label="", language="yaml", lines=25)

    # File upload handler
    def load_uploaded_file(file_obj):
        if file_obj is None:
            return python_hard
        try:
            with open(file_obj.name, "r", encoding="utf-8") as f:
                return f.read()
        except Exception as e:
            return f"Error reading file: {e}"

    file_upload.upload(fn=load_uploaded_file, inputs=file_upload, outputs=python_input)

    # Pipeline trigger
    generate_btn.click(
        fn=generate_artifacts,
        inputs=[python_input, model_dropdown],
        outputs=[rust_out, enhanced_out, tests_out, fastapi_out, openapi_out]
    )

ui.launch(inbrowser=True, share=False)
